## 📦 Cell 1: Geocoding Parking Violations with Mapbox
This cell samples 60,000 NYC parking violations, builds clean address strings, and geocodes them to latitude/longitude using the Mapbox API.
To avoid geocoding the same address multiple times, it maintains a local geocode_cache.json that stores results.
Coordinates are filtered to NYC bounds and exported to geocoded_fines_sample_50k.csv.

In [76]:
import pandas as pd
from mapbox import Geocoder
from tqdm import tqdm
import time
import os
import json

# ----------------------------------------
# CONFIGURATION
# ----------------------------------------
MAPBOX_TOKEN = "pk.eyJ1IjoiYXlhZmFoaW0iLCJhIjoiY21haDI0NzNtMDZnYjJrc2did2ozb2diMSJ9.ucfbzSq_1BEtyk7jqGJb1g"
CSV_PATH = "data/nyc_parking_violations_sample.csv"
CACHE_PATH = "geocode_cache.json"
BACKUP_PATH = "geocode_cache_backup.json"
SAVE_EVERY = 1000
SLEEP_TIME = 0.05
SAMPLE_SIZE = 60000  # Target: 60k unique addresses

# ----------------------------------------
# LOAD & CLEAN DATA
# ----------------------------------------
cols = ['house_number', 'street_name', 'violation_county']
df_raw = pd.read_csv(CSV_PATH, low_memory=False)
clean_df = df_raw.dropna(subset=cols).copy()

# Clean up address parts
clean_df['house_number'] = clean_df['house_number'].fillna('').astype(str).str.strip()
clean_df['street_name'] = clean_df['street_name'].fillna('').astype(str).str.strip()
clean_df['violation_county'] = clean_df['violation_county'].fillna('').astype(str).str.strip()

# Build full address
clean_df['full_address'] = (
    clean_df['house_number'] + " " +
    clean_df['street_name'] + ", " +
    clean_df['violation_county'] + ", NYC"
)

# Remove addresses that are too short or invalid
clean_df['full_address'] = clean_df['full_address'].str.replace('^\\s+', '', regex=True)
clean_df = clean_df[clean_df['full_address'].str.len() > 10]


# Shuffle to randomize address selection
clean_df = clean_df.sample(frac=1, random_state=42)

# Select 60k fines with unique addresses (1 per address)
seen_addresses = set()
selected_rows = []

for _, row in clean_df.iterrows():
    addr = row['full_address']
    if addr not in seen_addresses:
        selected_rows.append(row)
        seen_addresses.add(addr)
    if len(seen_addresses) >= SAMPLE_SIZE:
        break

df = pd.DataFrame(selected_rows)
print(f"📦 Selected {len(df)} fines with {len(df['full_address'].unique())} unique addresses")

# ----------------------------------------
# LOAD OR CREATE CACHE
# ----------------------------------------
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "r") as f:
        cache = json.load(f)
else:
    cache = {}

# ----------------------------------------
# SETUP GEOCODER
# ----------------------------------------
geocoder = Geocoder(access_token=MAPBOX_TOKEN)

# Filter uncached addresses
unique_addresses = df['full_address'].unique()
uncached = [addr for addr in unique_addresses if addr not in cache]
print(f"📍 Geocoding {len(uncached):,} new addresses...")

# ----------------------------------------
# GEOCODE LOOP
# ----------------------------------------
for i, addr in enumerate(tqdm(uncached)):
    try:
        response = geocoder.forward(addr, limit=1)
        features = response.geojson().get('features', [])
        if features:
            coords = features[0]['geometry']['coordinates']
            cache[addr] = [coords[1], coords[0]]  # lat, lon
        else:
            cache[addr] = [None, None]
    except Exception:
        cache[addr] = [None, None]
    time.sleep(SLEEP_TIME)

    # Save checkpoint
    if (i + 1) % SAVE_EVERY == 0:
        with open(CACHE_PATH, "w") as f:
            json.dump(cache, f)
        with open(BACKUP_PATH, "w") as f:
            json.dump(cache, f)
        print(f"💾 Checkpoint saved at {i+1} geocoded")

# Final cache save
with open(CACHE_PATH, "w") as f:
    json.dump(cache, f)
with open(BACKUP_PATH, "w") as f:
    json.dump(cache, f)
print("✅ Final geocode cache saved.")

# ----------------------------------------
# APPLY GEOCOORDINATES TO SAMPLE
# ----------------------------------------
df[['lat', 'lon']] = df['full_address'].apply(lambda x: pd.Series(cache.get(x, [None, None])))
df = df.dropna(subset=['lat', 'lon'])
df = df[(df['lat'].between(40.49, 40.92)) & (df['lon'].between(-74.26, -73.68))]

# ----------------------------------------
# MERGE COORDS INTO FULL ORIGINAL DATA
# ----------------------------------------
df_raw['full_address'] = (
    df_raw['house_number'].astype(str) + " " +
    df_raw['street_name'] + ", " +
    df_raw['violation_county'] + ", NYC"
)

# Create full_address in full dataset
df_raw['full_address'] = (
    df_raw['house_number'].astype(str) + " " +
    df_raw['street_name'] + ", " +
    df_raw['violation_county'] + ", NYC"
)

# Apply lat/lon from cache to ALL rows (not just sampled)
df_raw[['lat', 'lon']] = df_raw['full_address'].apply(lambda x: pd.Series(cache.get(x, [None, None])))

# Filter only valid NYC coordinates
final_df = df_raw.dropna(subset=['lat', 'lon'])
final_df = final_df[(final_df['lat'].between(40.49, 40.92)) & (final_df['lon'].between(-74.26, -73.68))]

# Save
final_df.to_csv("full_geocoded_parking_fines.csv", index=False)
print(f"✅ Final dataset saved with {len(final_df):,} geocoded rows.")

📦 Selected 58037 fines with 58037 unique addresses
📍 Geocoding 37,087 new addresses...


  2%|▏         | 635/37087 [02:22<2:16:44,  4.44it/s]


KeyboardInterrupt: 

In [77]:
import pandas as pd
import json
import os

# ----------------------------------------
# CONFIGURATION
# ----------------------------------------
CSV_PATH = "data/nyc_parking_violations_sample.csv"
CACHE_PATH = "geocode_cache.json"

# ----------------------------------------
# LOAD DATA & CACHE
# ----------------------------------------
df_raw = pd.read_csv(CSV_PATH, low_memory=False)

if not os.path.exists(CACHE_PATH):
    raise FileNotFoundError("Cache file not found. Run geocoder script first.")

with open(CACHE_PATH, "r") as f:
    cache = json.load(f)

# ----------------------------------------
# BUILD FULL ADDRESS FIELD
# ----------------------------------------
df_raw['house_number'] = df_raw['house_number'].fillna('').astype(str).str.strip()
df_raw['street_name'] = df_raw['street_name'].fillna('').astype(str).str.strip()
df_raw['violation_county'] = df_raw['violation_county'].fillna('').astype(str).str.strip()

df_raw['full_address'] = (
    df_raw['house_number'] + " " +
    df_raw['street_name'] + ", " +
    df_raw['violation_county'] + ", NYC"
).str.replace('^\\s+', '', regex=True)

# ----------------------------------------
# APPLY CACHED GEOCOORDINATES
# ----------------------------------------
df_raw[['lat', 'lon']] = df_raw['full_address'].apply(lambda x: pd.Series(cache.get(x, [None, None])))

# ----------------------------------------
# FILTER VALID NYC LOCATIONS
# ----------------------------------------
final_df = df_raw.dropna(subset=['lat', 'lon'])
final_df = final_df[
    (final_df['lat'].between(40.49, 40.92)) &
    (final_df['lon'].between(-74.26, -73.68))
]

# ----------------------------------------
# SAVE FINAL DATASET
# ----------------------------------------
final_df.to_csv("full_geocoded_parking_fines.csv", index=False)
print(f"✅ Final dataset saved with {len(final_df):,} geocoded rows (from cache only).")


✅ Final dataset saved with 49,878 geocoded rows (from cache only).


In [78]:
# Load the cache
with open("geocode_cache.json") as f:
    cache = json.load(f)

# Build full_address in your 80K dataset
df_raw = pd.read_csv("data/nyc_parking_violations_sample.csv", low_memory=False)
df_raw['full_address'] = (
    df_raw['house_number'].fillna('').astype(str).str.strip() + " " +
    df_raw['street_name'].fillna('').astype(str).str.strip() + ", " +
    df_raw['violation_county'].fillna('').astype(str).str.strip() + ", NYC"
)

# How many full_address values in df_raw match the cache?
matching = df_raw['full_address'].isin(cache.keys()).sum()
print(f"🎯 {matching:,} addresses in this dataset match the cache.")


🎯 50,384 addresses in this dataset match the cache.


## 🗺️ Cell 2: Spatial Join – Mapping Fines to Neighborhoods
This cell reads the geocoded fines and converts them into a GeoDataFrame.
It then loads the Neighborhood Tabulation Areas (NTA) from a GeoJSON file, reprojects it to the same coordinate system, and performs a spatial join to attach each fine to a neighborhood (if it intersects spatially).

In [79]:
import geopandas as gpd
import pandas as pd

# Load geocoded fine data
df = pd.read_csv("full_geocoded_parking_fines.csv", low_memory=False)

# Filter to NYC bounds (just in case)
# df = df[(df['lat'].between(40.49, 40.92)) & (df['lon'].between(-74.26, -73.68))]

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['lon'], df['lat']),
    crs="EPSG:4326"
)

# ✅ Load the clean GeoJSON instead of the shapefile
nta = gpd.read_file("data/nynta2020.geojson")  # replace path if needed
nta = nta.to_crs("EPSG:4326")  # ensure same CRS

# Spatial join: fines → neighborhoods
joined = gpd.sjoin(gdf, nta, how="left", predicate="intersects")


# Preview matched fines with neighborhood info
print(joined[['house_number', 'street_name', 'boroname', 'ntaname']].dropna().head())


  house_number     street_name   boroname                       ntaname
0           51       E 44TH ST  Manhattan          Midtown-Times Square
1        39-41         60TH ST     Queens                      Woodside
2       126 05          36 AVE     Queens                 College Point
3           41      SEAVER WAY     Queens  Flushing Meadows-Corona Park
4        46-50  BURLING STREET     Queens                 East Flushing


## 📊 Cell 3: Count Fines by Neighborhood
Here, we group the spatially joined data by ntaname (neighborhood name) and count how many fines occurred in each neighborhood.
This gives us raw counts of violations by area.

In [80]:
# Count number of fines per neighborhood
nta_counts = joined.groupby('ntaname').size().reset_index(name='num_fines')

# Preview top 10
print(nta_counts.sort_values(by='num_fines', ascending=False).head(10))


                                 ntaname  num_fines
63                          East Village        919
49   Downtown Brooklyn-DUMBO-Boerum Hill        893
134                 Midtown-Times Square        859
208            Upper West Side (Central)        798
58                   East Harlem (South)        775
34                  Chelsea-Hudson Yards        774
29                              Canarsie        771
57                   East Harlem (North)        763
91                     Greenwich Village        754
93                        Harlem (North)        747


## 👥 Cell 4: Normalize by Population
We load NYC population data by NTA and merge it with the fine data by matching on neighborhood names.
Then we compute fines per 1,000 residents, which normalizes the violation counts relative to population size — giving a fairer comparison across neighborhoods.

In [81]:
pop_df = pd.read_csv("data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv")

# Strip whitespace and normalize
joined['ntaname'] = joined['ntaname'].str.strip()
pop_df['NTA Name'] = pop_df['NTA Name'].str.strip()

# Merge using ntaname
merged = joined.merge(pop_df, left_on='ntaname', right_on='NTA Name', how='left')

# Group and compute stats
nta_stats = (
    merged.groupby(['ntaname', 'ntaabbrev', 'boroname', 'Population'])
    .size()
    .reset_index(name='num_fines')
)

nta_stats['fines_per_1000'] = (nta_stats['num_fines'] / nta_stats['Population']) * 1000

# Preview
print(nta_stats.sort_values(by='fines_per_1000', ascending=False).head())



              ntaname   ntaabbrev   boroname  Population  num_fines  \
38       East Village      EstVlg  Manhattan     41746.0        919   
24      College Point      CllgPt     Queens     21407.0        452   
39       East Village      EstVlg  Manhattan     44136.0        919   
25      College Point      CllgPt     Queens     24275.0        452   
28  Crotona Park East  CrtnaPkEst      Bronx     18072.0        334   

    fines_per_1000  
38       22.014085  
24       21.114589  
39       20.822005  
25       18.619979  
28       18.481629  


## 🧹 Cell 5: Clean and Aggregate Neighborhood Stats
Some neighborhoods may appear multiple times (due to data duplication or merged borders).
This step groups by neighborhood name again and computes the mean fines per 1,000 residents for each neighborhood to get a clean final dataset for visualization.

In [82]:
nta_cleaned = (
    nta_stats.groupby('ntaname')
    .agg({
        'num_fines': 'sum',
        'Population': 'mean',
        'fines_per_1000': 'mean'
    })
    .reset_index()
    .sort_values(by='fines_per_1000', ascending=False)
)
print(nta_cleaned.head())


                 ntaname  num_fines  Population  fines_per_1000
19          East Village       1838     42941.0       21.418045
12         College Point        904     22841.0       19.867284
14     Crotona Park East        668     19174.5       17.476747
51  Murray Hill-Kips Bay       1308     49580.5       13.197913
37               Jamaica       1266     52800.0       11.992527


In [83]:
# Save cleaned neighborhood-level stats
nta_cleaned.to_csv("data/nta_fine_stats_cleaned.csv", index=False)

# Also save raw joined fine points (optional)
joined.to_file("data/fines_joined_with_neighborhoods.geojson", driver="GeoJSON")

# Save GeoJSON with stats merged in (for choropleth)
geo_merged = nta.merge(nta_cleaned, on='ntaname', how='left')
geo_merged.to_file("data/nta_with_fine_stats.geojson", driver="GeoJSON")
